In [1]:
import sklearn
import pandas as pd
import numpy as np
import collections
import os.path
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.svm import SVC
from sklearn import linear_model
from sklearn import tree
from sklearn.model_selection import cross_val_score
from keras.utils import np_utils
from sklearn.neighbors import KNeighborsClassifier
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
import sys
from sklearn.ensemble import GradientBoostingRegressor
import math
import csv
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report, make_scorer, accuracy_score
from sklearn.calibration import CalibratedClassifierCV
import urllib
from sklearn.svm import LinearSVC
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, ParameterGrid
from datetime import datetime
import random
import warnings
from sklearn.tree import DecisionTreeClassifier, plot_tree

# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

In [2]:
import pandas as pd
import numpy as np
import os.path
import math
import collections

In [15]:
############################## LOAD CSVs ##############################

reg_season_compact_pd = pd.read_csv('Data/KaggleData_2023/MRegularSeasonCompactResults.csv')
teams_pd = pd.read_csv('Data/KaggleData_2023/MTeams.csv')
tourney_compact_pd = pd.read_csv('Data/KaggleData_2023/MNCAATourneyCompactResults.csv')
team_conferences_pd = pd.read_csv('Data/KaggleData_2023/MTeamConferences.csv')
tourney_results_pd = pd.read_csv('Data/KaggleData_2023/MTourneyResults.csv')
tourney_seeds_pd = pd.read_csv('Data/KaggleData_2023/MNCAATourneySeeds.csv')
conference_pd = pd.read_csv('Data/KaggleData/Conference.csv')



In [16]:
############################## DATA STRUCTURES ##############################

teamList = teams_pd['TeamName'].tolist()
NCAAChampionsList = tourney_results_pd['NCAA Champion'].tolist()

In [17]:
############################## HELPER FUNCTIONS ##############################

#change the year/season for the one you want to make predictions for
def checkPower6Conference(team_id):
    team_pd = team_conferences_pd[(team_conferences_pd['Season'] == 2023) & (team_conferences_pd['TeamID'] == team_id)]
    # Can't find the team
    if (len(team_pd) == 0):
        return 0
    confName = team_pd.iloc[0]['ConfAbbrev']
    return int(confName == 'sec' or confName == 'acc'or confName == 'big_ten' or confName == 'big_twelve' or confName == 'big_east' or confName == 'pac_twelve')

def getTeamID(name):
    return teams_pd[teams_pd['TeamName'] == name].values[0][0]

def getTeamName(team_id):
    return teams_pd[teams_pd['TeamID'] == team_id].values[0][1]

def getNumChampionships(team_id):
    name = getTeamName(team_id)
    return NCAAChampionsList.count(name)

def getListForURL(team_list):
    team_list = [x.lower() for x in team_list]
    team_list = [t.replace(' ', '-') for t in team_list]
    team_list = [t.replace('st', 'state') for t in team_list]
    team_list = [t.replace('northern-dakota', 'north-dakota') for t in team_list]
    team_list = [t.replace('nc-', 'north-carolina-') for t in team_list]
    team_list = [t.replace('fl-', 'florida-') for t in team_list]
    team_list = [t.replace('ga-', 'georgia-') for t in team_list]
    team_list = [t.replace('lsu', 'louisiana-state') for t in team_list]
    team_list = [t.replace('maristate', 'marist') for t in team_list]
    team_list = [t.replace('stateate', 'state') for t in team_list]
    team_list = [t.replace('northernorthern', 'northern') for t in team_list]
    team_list = [t.replace('usc', 'southern-california') for t in team_list]
    base = 'http://www.sports-reference.com/cbb/schools/'
    for team in team_list:
        url = base + team + '/'
getListForURL(teamList);

def handleCases(arr):
    indices = []
    listLen = len(arr)
    for i in range(listLen):
        if (arr[i] == 'St' or arr[i] == 'FL'):
            indices.append(i)
    for p in indices:
        arr[p-1] = arr[p-1] + ' ' + arr[p]
    for i in range(len(indices)): 
        arr.remove(arr[indices[i] - i])
    return arr

def checkConferenceChamp(team_id, year):
    year_conf_pd = conference_pd[conference_pd['Year'] == year]
    champs = year_conf_pd['Regular Season Champ'].tolist()
    # For handling cases where there is more than one champion
    champs_separated = [words for segments in champs for words in segments.split()]
    name = getTeamName(team_id)
    champs_separated = handleCases(champs_separated)
    if (name in champs_separated):
        return 1
    else:
        return 0

def checkConferenceTourneyChamp(team_id, year):
    year_conf_pd = conference_pd[conference_pd['Year'] == year]
    champs = year_conf_pd['Tournament Champ'].tolist()
    name = getTeamName(team_id)
    if (name in champs):
        return 1
    else:
        return 0

def getTourneyAppearances(team_id):
    return len(tourney_seeds_pd[tourney_seeds_pd['TeamID'] == team_id].index)

def handleDifferentCSV(df):
    # The stats CSV is a lit different in terms of naming so below is just some data cleaning
    df['School'] = df['School'].replace('(State)', 'St', regex=True) 
    df['School'] = df['School'].replace('Albany (NY)', 'Albany NY') 
    df['School'] = df['School'].replace('Boston University', 'Boston Univ')
    df['School'] = df['School'].replace('Central Michigan', 'C Michigan')
    df['School'] = df['School'].replace('(Eastern)', 'E', regex=True)
    df['School'] = df['School'].replace('Louisiana St', 'LSU')
    df['School'] = df['School'].replace('North Carolina St', 'NC State')
    df['School'] = df['School'].replace('Southern California', 'USC')
    df['School'] = df['School'].replace('University of California', 'California', regex=True) 
    df['School'] = df['School'].replace('American', 'American Univ')
    df['School'] = df['School'].replace('Arkansas-Little Rock', 'Ark Little Rock')
    df['School'] = df['School'].replace('Arkansas-Pine Bluff', 'Ark Pine Bluff')
    df['School'] = df['School'].replace('Bowling Green St', 'Bowling Green')
    df['School'] = df['School'].replace('Brigham Young', 'BYU')
    df['School'] = df['School'].replace('Cal Poly', 'Cal Poly SLO')
    df['School'] = df['School'].replace('Centenary (LA)', 'Centenary')
    df['School'] = df['School'].replace('Central Connecticut St', 'Central Conn')
    df['School'] = df['School'].replace('Charleston Southern', 'Charleston So')
    df['School'] = df['School'].replace('Coastal Carolina', 'Coastal Car')
    df['School'] = df['School'].replace('College of Charleston', 'Col Charleston')
    df['School'] = df['School'].replace('Cal St Fullerton', 'CS Fullerton')
    df['School'] = df['School'].replace('Cal St Sacramento', 'CS Sacramento')
    df['School'] = df['School'].replace('Cal St Bakersfield', 'CS Bakersfield')
    df['School'] = df['School'].replace('Cal St Northridge', 'CS Northridge')
    df['School'] = df['School'].replace('East Tennessee St', 'ETSU')
    df['School'] = df['School'].replace('Detroit Mercy', 'Detroit')
    df['School'] = df['School'].replace('Fairleigh Dickinson', 'F Dickinson')
    df['School'] = df['School'].replace('Florida Atlantic', 'FL Atlantic')
    df['School'] = df['School'].replace('Florida Gulf Coast', 'FL Gulf Coast')
    df['School'] = df['School'].replace('Florida International', 'Florida Intl')
    df['School'] = df['School'].replace('George Washington', 'G Washington')
    df['School'] = df['School'].replace('Georgia Southern', 'Ga Southern')
    df['School'] = df['School'].replace('Gardner-Webb', 'Gardner Webb')
    df['School'] = df['School'].replace('Illinois-Chicago', 'IL Chicago')
    df['School'] = df['School'].replace('Kent St', 'Kent')
    df['School'] = df['School'].replace('Long Island University', 'Long Island')
    df['School'] = df['School'].replace('Loyola Marymount', 'Loy Marymount')
    df['School'] = df['School'].replace('Loyola (MD)', 'Loyola MD')
    df['School'] = df['School'].replace('Loyola (IL)', 'Loyola-Chicago')
    df['School'] = df['School'].replace('Massachusetts', 'MA Lowell')
    df['School'] = df['School'].replace('Maryland-Eastern Shore', 'MD E Shore')
    df['School'] = df['School'].replace('Miami (FL)', 'Miami FL')
    df['School'] = df['School'].replace('Miami (OH)', 'Miami OH')
    df['School'] = df['School'].replace('Missouri-Kansas City', 'Missouri KC')
    df['School'] = df['School'].replace('Monmouth', 'Monmouth NJ')
    df['School'] = df['School'].replace('Mississippi Valley St', 'MS Valley St')
    df['School'] = df['School'].replace('Montana St', 'MTSU')
    df['School'] = df['School'].replace('Northern Colorado', 'N Colorado')
    df['School'] = df['School'].replace('North Dakota St', 'N Dakota St')
    df['School'] = df['School'].replace('Northern Illinois', 'N Illinois')
    df['School'] = df['School'].replace('Northern Kentucky', 'N Kentucky')
    df['School'] = df['School'].replace('North Carolina A&T', 'NC A&T')
    df['School'] = df['School'].replace('North Carolina Central', 'NC Central')
    df['School'] = df['School'].replace('Pennsylvania', 'Penn')
    df['School'] = df['School'].replace('South Carolina St', 'S Carolina St')
    df['School'] = df['School'].replace('Southern Illinois', 'S Illinois')
    df['School'] = df['School'].replace('UC-Santa Barbara', 'Santa Barbara')
    df['School'] = df['School'].replace('Southeastern Louisiana', 'SE Louisiana')
    df['School'] = df['School'].replace('Southeast Missouri St', 'SE Missouri St')
    df['School'] = df['School'].replace('Stephen F. Austin', 'SF Austin')
    df['School'] = df['School'].replace('Southern Methodist', 'SMU')
    df['School'] = df['School'].replace('Southern Mississippi', 'Southern Miss')
    df['School'] = df['School'].replace('Southern', 'Southern Univ')
    df['School'] = df['School'].replace('St. Bonaventure', 'St Bonaventure')
    df['School'] = df['School'].replace('St. Francis (NY)', 'St Francis NY')
    df['School'] = df['School'].replace('Saint Francis (PA)', 'St Francis PA')
    df['School'] = df['School'].replace('St. John\'s (NY)', 'St John\'s')
    df['School'] = df['School'].replace('Saint Joseph\'s', 'St Joseph\'s PA')
    df['School'] = df['School'].replace('Saint Louis', 'St Louis')
    df['School'] = df['School'].replace('Saint Mary\'s (CA)', 'St Mary\'s CA')
    df['School'] = df['School'].replace('Mount Saint Mary\'s', 'Mt St Mary\'s')
    df['School'] = df['School'].replace('Saint Peter\'s', 'St Peter\'s')
    df['School'] = df['School'].replace('Texas A&M-Corpus Christian', 'TAM C. Christian')
    df['School'] = df['School'].replace('Texas Christian', 'TCU')
    df['School'] = df['School'].replace('Tennessee-Martin', 'TN Martin')
    df['School'] = df['School'].replace('Texas-Rio Grande Valley', 'UTRGV')
    df['School'] = df['School'].replace('Texas Southern', 'TX Southern')
    df['School'] = df['School'].replace('Alabama-Birmingham', 'UAB')
    df['School'] = df['School'].replace('UC-Davis', 'UC Davis')
    df['School'] = df['School'].replace('UC-Irvine', 'UC Irvine')
    df['School'] = df['School'].replace('UC-Riverside', 'UC Riverside')
    df['School'] = df['School'].replace('Central Florida', 'UCF')
    df['School'] = df['School'].replace('Louisiana-Lafayette', 'ULL')
    df['School'] = df['School'].replace('Louisiana-Monroe', 'ULM')
    df['School'] = df['School'].replace('Maryland-Baltimore County', 'UMBC')
    df['School'] = df['School'].replace('North Carolina-Asheville', 'UNC Asheville')
    df['School'] = df['School'].replace('North Carolina-Greensboro', 'UNC Greensboro')
    df['School'] = df['School'].replace('North Carolina-Wilmington', 'UNC Wilmington')
    df['School'] = df['School'].replace('Nevada-Las Vegas', 'UNLV')
    df['School'] = df['School'].replace('Texas-Arlington', 'UT Arlington')
    df['School'] = df['School'].replace('Texas-San Antonio', 'UT San Antonio')
    df['School'] = df['School'].replace('Texas-El Paso', 'UTEP')
    df['School'] = df['School'].replace('Virginia Commonwealth', 'VA Commonwealth')
    df['School'] = df['School'].replace('Western Carolina', 'W Carolina')
    df['School'] = df['School'].replace('Western Illinois', 'W Illinois')
    df['School'] = df['School'].replace('Western Kentucky', 'WKU')
    df['School'] = df['School'].replace('Western Michigan', 'W Michigan')
    df['School'] = df['School'].replace('Abilene Christian', 'Abilene Chr')
    df['School'] = df['School'].replace('Montana State', 'Montana St')
    df['School'] = df['School'].replace('Central Arkansas', 'Cent Arkansas')
    df['School'] = df['School'].replace('Houston Baptist', 'Houston Bap')
    df['School'] = df['School'].replace('South Dakota St', 'S Dakota St')
    df['School'] = df['School'].replace('Maryland-Eastern Shore', 'MD E Shore')
    return df

def getHomeStat(row):
    if (row == 'H'):
        home = 1
    if (row == 'A'):
        home = -1
    if (row == 'N'):
        home = 0
    return home

def compareTwoTeams(id_1, id_2, year):
    team_1 = getSeasonData(id_1, year)
    team_2 = getSeasonData(id_2, year)
    diff = [a - b for a, b in zip(team_1, team_2)]
    return diff

def normalizeInput(arr):
    for i in range(arr.shape[1]):
        minVal = min(arr[:,i])
        maxVal = max(arr[:,i])
        arr[:,i] =  (arr[:,i] - minVal) / (maxVal - minVal)
    return arr

def normalizeInput2(X):
    return (X - np.mean(X, axis = 0)) / np.std(X, axis = 0)

In [18]:
############################## MAIN PREPROCESSING FUNCTIONS ##############################

def getSeasonData(team_id, year):
    stats_SOS_pd = pd.read_csv('Data/RegSeasonStats/MMStats_'+str(year)+'.csv', encoding= 'unicode_escape')
    stats_SOS_pd = handleDifferentCSV(stats_SOS_pd)
    ratings_pd = pd.read_csv('Data/RatingStats/RatingStats_'+str(year)+'.csv', encoding= 'unicode_escape')
    ratings_pd = handleDifferentCSV(ratings_pd)
    year_data_pd = reg_season_compact_pd[reg_season_compact_pd['Season'] == year]
    
    numFeatures = 20 #22
    name = getTeamName(team_id)
    team = stats_SOS_pd[stats_SOS_pd['School'] == name]
    team_rating = ratings_pd[ratings_pd['School'] == name]
    if (len(team.index) == 0 or len(team_rating.index) == 0): #Can't find the team
        return [0 for x in range(numFeatures)]

    gamesWon = team['W'].values[0]
    gamesLost = team['L'].values[0]
    total3sMade = team['X3P'].values[0]
    totalTurnovers = 0 if math.isnan(team['TOV'].values[0]) else team['TOV'].values[0]
    totalAssists = 0 if math.isnan(team['AST'].values[0]) else team['AST'].values[0]
    totalRebounds = 0 if math.isnan(team['TRB'].values[0]) else team['TRB'].values[0]
    totalSteals = 0 if math.isnan(team['STL'].values[0]) else team['STL'].values[0]
    totalOffRebounds = 0 if math.isnan(team['ORB'].values[0]) else team['ORB'].values[0]
    totalBlocks = 0 if math.isnan(team['BLK'].values[0]) else team['BLK'].values[0]
    fgPercent = 0 if math.isnan(team['FG.'].values[0]) else team['FG.'].values[0]
    x3ptPercent = 0 if math.isnan(team['X3P.'].values[0]) else team['X3P.'].values[0]
    fgAttempts = 0 if math.isnan(team['FGA'].values[0]) else team['FGA'].values[0]
    ftAttemps = 0 if math.isnan(team['FTA'].values[0]) else team['FTA'].values[0]
    nrtg =  0 if math.isnan(team_rating['NRtg'].values[0]) else team_rating['NRtg'].values[0]
    
    
    sos = team['SOS'].values[0]
    srs = team['SRS'].values[0]
    numWins = team['W'].values[0]
    totalPointsScored = team['Tm.'].values[0]

    totalPointsAllowed = team['Opp.'].values[0]
    # MM_Stats 1993-1995 don't have these stats so we need to get it from somewhere else
    if math.isnan(totalPointsAllowed):
        gamesPlayed = year_data_pd[(year_data_pd.WTeamID == team_id) | (year_data_pd.LTeamID == team_id)] 
        totalPointsAllowed = gamesPlayed['LScore'].sum()
    
    #Finding tournament seed for that year
    tourneyYear = tourney_seeds_pd[tourney_seeds_pd['Season'] == year]
    seed = tourneyYear[tourneyYear['TeamID'] == team_id]
    if (len(seed.index) != 0):
        seed = seed.values[0][1]
        tournamentSeed = int(seed[1:3])
    else:
        tournamentSeed = 25 #Not sure how to represent if a team didn't make the tourney
    
    numGames = team['G'].values[0]

    avgPointsScored = totalPointsScored/numGames
    avgPointsAllowed = totalPointsAllowed/numGames
    avg3sMade = total3sMade/numGames
    avgTurnovers = totalTurnovers/numGames
    avgAssists = totalAssists/numGames
    avgRebounds = totalRebounds/numGames
    avgSteals = totalSteals/numGames
    avgBlocks = totalBlocks/numGames
    totalPoss = (fgAttempts-totalOffRebounds+totalTurnovers+(0.4*ftAttemps))
    offEff = totalPointsScored/totalPoss
    defEff = totalPointsAllowed/totalPoss
    #total number of features to return
    return [numWins, avgPointsScored, avgPointsAllowed, checkPower6Conference(team_id), avg3sMade, avgAssists, avgTurnovers,
           checkConferenceChamp(team_id, year), checkConferenceTourneyChamp(team_id, year), tournamentSeed,
            sos, srs, avgRebounds, avgSteals, getTourneyAppearances(team_id), getNumChampionships(team_id),
            offEff,defEff,avgBlocks, nrtg]

def createSeasonDict(year):
    seasonDictionary = collections.defaultdict(list)
    for team in teamList:
        team_id = teams_pd[teams_pd['TeamName'] == team].values[0][0]
        team_vector = getSeasonData(team_id, year)
        seasonDictionary[team_id] = team_vector
    return seasonDictionary

  



In [19]:
def createTrainingSet(years, saveYears):
    totalNumGames = 0
    for year in years:
        season = reg_season_compact_pd[reg_season_compact_pd['Season'] == year]
        totalNumGames += len(season.index)
        tourney = tourney_compact_pd[tourney_compact_pd['Season'] == year]
        totalNumGames += len(tourney.index)
    numFeatures = len(getSeasonData(1181,2012)) #Just choosing a random team and seeing the dimensionality of the vector
    xTrain = np.zeros(( totalNumGames, numFeatures + 1))
    yTrain = np.zeros(( totalNumGames ))
    indexCounter = 0
    for year in years:
        team_vectors = createSeasonDict(year)
        season = reg_season_compact_pd[reg_season_compact_pd['Season'] == year]
        numGamesInSeason = len(season.index)
        tourney = tourney_compact_pd[tourney_compact_pd['Season'] == year]
        numGamesInSeason += len(tourney.index)
        xTrainSeason = np.zeros(( numGamesInSeason, numFeatures + 1))
        yTrainSeason = np.zeros(( numGamesInSeason ))
        counter = 0
        for index, row in season.iterrows():
            w_team = row['WTeamID']
            w_vector = team_vectors[w_team]
            l_team = row['LTeamID']
            l_vector = team_vectors[l_team]
            diff = [a - b for a, b in zip(w_vector, l_vector)]
            home = getHomeStat(row['WLoc'])
            if (counter % 2 == 0):
                diff.append(home) 
                xTrainSeason[counter] = diff
                yTrainSeason[counter] = 1
            else:
                diff.append(-home)
                xTrainSeason[counter] = [ -p for p in diff]
                yTrainSeason[counter] = 0
            counter += 1
        for index, row in tourney.iterrows():
            w_team = row['WTeamID']
            w_vector = team_vectors[w_team]
            l_team = row['LTeamID']
            l_vector = team_vectors[l_team]
            diff = [a - b for a, b in zip(w_vector, l_vector)]
            home = 0 #All tournament games are neutral
            if (counter % 2 == 0):
                diff.append(home) 
                xTrainSeason[counter] = diff
                yTrainSeason[counter] = 1
            else:
                diff.append(-home)
                xTrainSeason[counter] = [ -p for p in diff]
                yTrainSeason[counter] = 0
            counter += 1
        xTrain[indexCounter:numGamesInSeason+indexCounter] = xTrainSeason
        yTrain[indexCounter:numGamesInSeason+indexCounter] = yTrainSeason
        indexCounter += numGamesInSeason
        print ('Finished year:', year)
        if (year in saveYears):
            np.save('Data/PrecomputedMatrices/TeamVectors/' + str(year) + 'TeamVectors', team_vectors)
    return xTrain, yTrain

def createAndSave(years, saveYears):
    xTrain, yTrain = createTrainingSet(years, saveYears)
    print ("Shape of xTrain:", xTrain.shape)
    print ("Shape of yTrain:", yTrain.shape)
    np.save('Data/PrecomputedMatrices/xTrain', xTrain)
    np.save('Data/PrecomputedMatrices/yTrain', yTrain)    

In [20]:
############################## CREATE TRAINING SET ##############################

# In case you want to run with Python 2
try:
    input = raw_input
except NameError:
    pass

endYear = int(input('What year do you have data until?\n'))

years = range(2015,endYear + 1)
# Saves the team vectors for the following years
saveYears = range(endYear - 4,endYear + 1)
if os.path.exists("Data/PrecomputedMatrices/xTrain.npy") and os.path.exists("Data/PrecomputedMatrices/yTrain.npy"):
    print ('There is already a precomputed xTrain and yTrain.')
    response = input('Do you want to remove these files and create a new training set? (y/n) ')
    if (response == 'y'):
        os.remove("Data/PrecomputedMatrices/xTrain.npy")
        os.remove("Data/PrecomputedMatrices/yTrain.npy")
        createAndSave(years, saveYears)
    else: 
        print ('Okay, going to exit now.')
else:
    createAndSave(years, saveYears)

What year do you have data until?
2023
There is already a precomputed xTrain and yTrain.
Do you want to remove these files and create a new training set? (y/n) y
Finished year: 2015
Finished year: 2016
Finished year: 2017
Finished year: 2018
Finished year: 2019
Finished year: 2020


C:\Users\fshausb\AppData\Local\Temp\ipykernel_28772\2426079409.py:56: RuntimeWarning: invalid value encountered in longlong_scalars
  avgPointsAllowed = totalPointsAllowed/numGames
C:\Users\fshausb\AppData\Local\Temp\ipykernel_28772\2426079409.py:58: RuntimeWarning: invalid value encountered in longlong_scalars
  avgTurnovers = totalTurnovers/numGames
C:\Users\fshausb\AppData\Local\Temp\ipykernel_28772\2426079409.py:59: RuntimeWarning: invalid value encountered in longlong_scalars
  avgAssists = totalAssists/numGames
C:\Users\fshausb\AppData\Local\Temp\ipykernel_28772\2426079409.py:60: RuntimeWarning: invalid value encountered in longlong_scalars
  avgRebounds = totalRebounds/numGames
C:\Users\fshausb\AppData\Local\Temp\ipykernel_28772\2426079409.py:61: RuntimeWarning: invalid value encountered in longlong_scalars
  avgSteals = totalSteals/numGames
C:\Users\fshausb\AppData\Local\Temp\ipykernel_28772\2426079409.py:62: RuntimeWarning: invalid value encountered in longlong_scalars
  avgBl

C:\Users\fshausb\AppData\Local\Temp\ipykernel_28772\2426079409.py:56: RuntimeWarning: invalid value encountered in longlong_scalars
  avgPointsAllowed = totalPointsAllowed/numGames
C:\Users\fshausb\AppData\Local\Temp\ipykernel_28772\2426079409.py:58: RuntimeWarning: invalid value encountered in longlong_scalars
  avgTurnovers = totalTurnovers/numGames
C:\Users\fshausb\AppData\Local\Temp\ipykernel_28772\2426079409.py:59: RuntimeWarning: invalid value encountered in longlong_scalars
  avgAssists = totalAssists/numGames
C:\Users\fshausb\AppData\Local\Temp\ipykernel_28772\2426079409.py:60: RuntimeWarning: invalid value encountered in longlong_scalars
  avgRebounds = totalRebounds/numGames
C:\Users\fshausb\AppData\Local\Temp\ipykernel_28772\2426079409.py:61: RuntimeWarning: invalid value encountered in longlong_scalars
  avgSteals = totalSteals/numGames
C:\Users\fshausb\AppData\Local\Temp\ipykernel_28772\2426079409.py:62: RuntimeWarning: invalid value encountered in longlong_scalars
  avgBl

Finished year: 2021
Finished year: 2022
Finished year: 2023
Shape of xTrain: (46784, 21)
Shape of yTrain: (46784,)


In [21]:
############################## LOAD TRAINING SET ##############################

if os.path.exists("Data/PrecomputedMatrices/xTrain.npy") and os.path.exists("Data/PrecomputedMatrices/yTrain.npy"):
	xTrain = np.load("Data/PrecomputedMatrices/xTrain.npy")
	yTrain = np.load("Data/PrecomputedMatrices/yTrain.npy")
	print ("Shape of xTrain:", xTrain.shape)
	print ("Shape of yTrain:", yTrain.shape)
else:
	print ('We need a training set! Run dataPreprocessing.py')
	sys.exit()

# In case you want to run with Python 2
try:
    input = raw_input
except NameError:
    pass

curYear = int(input('What year are these predictions for?\n'))

############################## LOAD CSV FILES ##############################

sample_sub_pd = pd.read_csv('Data/KaggleData/SampleSubmissionStage1.csv')
sample_sub_pd2 = pd.read_csv('Data/KaggleData/SampleSubmissionStage2.csv')
teams_pd = pd.read_csv('Data/KaggleData/Teams.csv')


Shape of xTrain: (46784, 21)
Shape of yTrain: (46784,)
What year are these predictions for?
2023


In [22]:
############################## TRAIN MODEL ##############################

model = GradientBoostingRegressor(n_estimators=35, max_depth=7, criterion='squared_error', 
                                  learning_rate = 0.15,warm_start=True)

categories=['Wins','PPG','PPGA','PowerConf','3PG', 'APG','TOP','Conference Champ','Tourney Conference Champ',
           'Seed','SOS','SRS', 'RPG', 'SPG', 'Tourney Appearances','National Championships','Location','offEff','defEff']
accuracy=[]
numTrials = 3

for i in range(numTrials):
    X_train, X_test, Y_train, Y_test = train_test_split(xTrain, yTrain, test_size= 0.25)
    startTime = datetime.now() # For some timing stuff
    results = model.fit(X_train, Y_train)
    preds = model.predict(X_test)

    preds[preds < .5] = 0
    preds[preds >= .5] = 1
    localAccuracy = np.mean(preds == Y_test)
    accuracy.append(localAccuracy)
    print ("Finished run #" + str(i) + ". Accuracy = " + str(localAccuracy))
    print ("Time taken: " + str(datetime.now() - startTime))
if numTrials != 0:
	print ("The average accuracy is", sum(accuracy)/len(accuracy))

Finished run #0. Accuracy = 0.7265731874145007
Time taken: 0:00:12.474468
Finished run #1. Accuracy = 0.7731703146374829
Time taken: 0:00:00.047000
Finished run #2. Accuracy = 0.7707763337893296
Time taken: 0:00:00.048000
The average accuracy is 0.7568399452804377


In [45]:
############################## TRAIN MODEL ##############################

model = GradientBoostingRegressor(n_estimators=350, max_depth=7, learning_rate = 0.1,warm_start=True)

categories=['Wins','PPG','PPGA','PowerConf','3PG', 'APG','TOP','Conference Champ','Tourney Conference Champ',
           'Seed','SOS','SRS', 'RPG', 'SPG', 'Tourney Appearances','National Championships','Location','offEff','defEff']
accuracy=[]
numTrials = 3

for i in range(numTrials):
    X_train, X_test, Y_train, Y_test = train_test_split(xTrain, yTrain, test_size= 0.25)
    startTime = datetime.now() # For some timing stuff
    results = model.fit(X_train, Y_train)
    preds = model.predict(X_test)

    preds[preds < .5] = 0
    preds[preds >= .5] = 1
    localAccuracy = np.mean(preds == Y_test)
    accuracy.append(localAccuracy)
    print ("Finished run #" + str(i) + ". Accuracy = " + str(localAccuracy))
    print ("Time taken: " + str(datetime.now() - startTime))
if numTrials != 0:
	print ("The average accuracy is", sum(accuracy)/len(accuracy))

Finished run #0. Accuracy = 0.7189583333333334
Time taken: 0:03:04.857740
Finished run #1. Accuracy = 0.8242361111111111
Time taken: 0:00:00.534039
Finished run #2. Accuracy = 0.8290972222222223
Time taken: 0:00:00.558191
The average accuracy is 0.7907638888888889


In [30]:
#Testing out linearRegression model, this doesnt yeild great results

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing, svm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

regr = LinearRegression()
regr.fit(X_train, Y_train)
print(regr.score(X_test, Y_test))

0.26530643679075094


In [32]:
############################## TRAIN MODEL Random Forest ##############################

# creating a RF classifier
clf = RandomForestClassifier(n_estimators = 800, max_depth=7) 

# fit function is used to train the model using the training sets as parameters
clf.fit(X_train, Y_train)

  
# performing predictions on the test dataset
y_pred = clf.predict(X_test)
  
# metrics are used to find accuracy or error
from sklearn import metrics  
print()
  
# using metrics module for accuracy calculation
print("ACCURACY OF THE MODEL: ", metrics.accuracy_score(Y_test, y_pred))


ACCURACY OF THE MODEL:  0.724093707250342


In [33]:
# random forest with random undersampling for imbalanced classification
from numpy import mean
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from imblearn.ensemble import BalancedRandomForestClassifier
# generate dataset

# define model
model1 = BalancedRandomForestClassifier(n_estimators=100, criterion='entropy',warm_start = True, ccp_alpha=0.015)
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(model1, X_train, Y_train, scoring='roc_auc', cv=cv, n_jobs=-1)
# summarize performance
print('Mean ROC AUC: %.3f' % mean(scores))


Mean ROC AUC: 0.792


In [1]:
#Testing out DecisionTree model, this doesnt yeild great results

DT = DecisionTreeClassifier(random_state=42)

DT.fit(X_train, Y_train)

print(classification_report(Y_test, DT.predict(X_test)))

max_depth = DT.get_depth()



NameError: name 'DecisionTreeClassifier' is not defined

In [36]:
max_depth_grid_search = GridSearchCV(estimator=DecisionTreeClassifier(random_state=42),scoring=make_scorer(accuracy_score)
                                     ,param_grid=ParameterGrid({"max_depth":[[max_depth] for max_depth in range (1, max_depth +1)]}))


max_depth_grid_search.fit(X_train,Y_train)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42),
             param_grid=<sklearn.model_selection._search.ParameterGrid object at 0x0000028ADB664FD0>,
             scoring=make_scorer(accuracy_score))

In [37]:
max_depth_grid_search.best_params_

{'max_depth': 7}

In [41]:
DT2 = DecisionTreeClassifier(random_state=42)

DT2.fit(xTrain, yTrain)

print(classification_report(Y_test, DT.predict(X_test)))

max_depth2 = DT.get_depth()


              precision    recall  f1-score   support

         0.0       0.89      0.98      0.93     12848
         1.0       0.98      0.88      0.92     13003

    accuracy                           0.93     25851
   macro avg       0.93      0.93      0.93     25851
weighted avg       0.93      0.93      0.93     25851



In [42]:
max_depth2

49

In [44]:
ccp_alphas = DT2.cost_complexity_pruning_path(xTrain,yTrain)["ccp_alphas"]

In [45]:
ccp_alphas

array([ 0.00000000e+00, -6.77626358e-21, -6.77626358e-21, ...,
        1.24970342e-02,  1.56004496e-02,  9.32172404e-02])

In [48]:
ccp_alpha_grid_search = GridSearchCV(
estimator=DecisionTreeClassifier(random_state=42), scoring=make_scorer(accuracy_score), 
    param_grid=ParameterGrid({"ccp_alpha":[[alpha] for alpha in ccp_alphas]}))

In [ ]:
#This takes forever to run and probably isnt worth the time
ccp_alpha_grid_search.fit(xTrain,yTrain)

In [ ]:
#this wont run unless the cell above completes
ccp_alpha_grid_search.best_params_

In [11]:
############################## TEST MODEL ##############################

#this needs to run before making current year predictions

def predictGame(team_1_vector, team_2_vector, home, modelUsed):
    diff = [a - b for a, b in zip(team_1_vector, team_2_vector)]
    diff.append(home)
    if hasattr(modelUsed, 'predict_proba'):
	    return modelUsed.predict_proba([diff])[0][1]
    elif hasattr(modelUsed, 'predict'):
        return modelUsed.predict([diff])[0]
    else:
        raise AttributeError("Model does not have expected prediction method")




In [23]:
############################## CREATE KAGGLE SUBMISSION ##############################
#this needs to run before making current year predictions


def loadTeamVectors(years):
	listDictionaries = []
	for year in years:
		curVectors = np.load("Data/PrecomputedMatrices/TeamVectors/" + str(year) + "TeamVectors.npy",allow_pickle=True).item()
		listDictionaries.append(curVectors)
	return listDictionaries



In [24]:
#this needs to run before making current year predictions
def createPrediction(stage2 = False):
	if stage2:
		years = [curYear]
		localPd = sample_sub_pd2
	else:
		# The years that we want to predict for
		years = range(curYear - 4,curYear)
		localPd = sample_sub_pd

	if os.path.exists("result.csv"):
		os.remove("result.csv")
	listDictionaries = loadTeamVectors(years)
	print ("Loaded the team vectors")
	results = [[0 for x in range(2)] for x in range(len(localPd.index))]

	predictionModel = GradientBoostingRegressor(n_estimators=100, max_depth=5)
	predictionModel.fit(xTrain, yTrain)

	for index, row in localPd.iterrows():
		matchupId = row['ID']
		year = int(matchupId[0:4]) 
		teamVectors = listDictionaries[year - years[0]]
		team1Id = int(matchupId[5:9])
		team2Id = int(matchupId[10:14])
		team1Vector = teamVectors[team1Id] 
		team2Vector = teamVectors[team2Id]
		pred1 = predictGame(team1Vector, team2Vector, 0, predictionModel)
		pred = pred1.clip(0.,1.)
		results[index][0] = matchupId
		results[index][1] = pred
	results = pd.np.array(results)
	firstRow = [[0 for x in range(2)] for x in range(1)]
	firstRow[0][0] = 'ID'
	firstRow[0][1] = 'Pred'
	with open("result.csv", "w") as f:
		writer = csv.writer(f)
		writer.writerows(firstRow)
		writer.writerows(results)

#createPrediction()
createPrediction(stage2=True)

Loaded the team vectors


IndexError: list index out of range

In [21]:
############################## TRAIN MODEL ##############################

model = GradientBoostingRegressor(n_estimators=350, max_depth=7, criterion='squared_error', learning_rate = 0.15,warm_start=True)

categories=['Wins','PPG','PPGA','PowerConf','3PG', 'APG','TOP','Conference Champ','Tourney Conference Champ',
           'Seed','SOS','SRS', 'RPG', 'SPG', 'Tourney Appearances','National Championships','Location','offEff','defEff']
accuracy=[]
numTrials = 3

for i in range(numTrials):
    X_train, X_test, Y_train, Y_test = train_test_split(xTrain, yTrain, test_size= 0.25)
    startTime = datetime.now() # For some timing stuff
    results = model.fit(X_train, Y_train)
    preds = model.predict(X_test)

    preds[preds < .5] = 0
    preds[preds >= .5] = 1
    localAccuracy = np.mean(preds == Y_test)
    accuracy.append(localAccuracy)
    print ("Finished run #" + str(i) + ". Accuracy = " + str(localAccuracy))
    print ("Time taken: " + str(datetime.now() - startTime))
if numTrials != 0:
	print ("The average accuracy is", sum(accuracy)/len(accuracy))

Finished run #0. Accuracy = 0.7079861111111111
Time taken: 0:04:09.065086
Finished run #1. Accuracy = 0.8411805555555556
Time taken: 0:00:00.572056
Finished run #2. Accuracy = 0.8394444444444444
Time taken: 0:00:00.547927
The average accuracy is 0.7962037037037039


In [56]:

############################## PREDICTING THIS YEAR'S BRACKET ##############################
#round 1 hoftstra bracket
def trainModel():
    model = GradientBoostingRegressor(n_estimators=350, max_depth=7, learning_rate = 0.1,warm_start=True)
    numTrials = 3
    for i in range(numTrials):
        X_train, X_test, Y_train, Y_test = train_test_split(xTrain, yTrain, test_size= 0.25)
        startTime = datetime.now() # For some timing stuff
        results = model.fit(X_train, Y_train)
        preds = model.predict(X_test)

        preds[preds < .5] = 0
        preds[preds >= .5] = 1
        localAccuracy = np.mean(preds == Y_test)
        accuracy.append(localAccuracy)
        print ("Finished run #" + str(i) + ". Accuracy = " + str(localAccuracy))
        print ("Time taken: " + str(datetime.now() - startTime))

    if numTrials != 0:
        print ("The average accuracy is", sum(accuracy)/len(accuracy))
        
    results.fit(xTrain, yTrain)

    return model

def findWinner(team1, team2, modelUsed):
    year = [curYear]
    teamVectors = loadTeamVectors(year)[0]
    team1Vector = teamVectors[int(teams_pd[teams_pd['TeamName'] == team1].values[0][0])]
    team2Vector = teamVectors[int(teams_pd[teams_pd['TeamName'] == team2].values[0][0])]
    prediction = predictGame(team1Vector, team2Vector, 0, modelUsed)
    if (prediction < 0.5):
        print ("Probability that {0} wins: {1}".format(team2, 1 - prediction))
    else:
        print ("Probability that {0} wins: {1}".format(team1, prediction))



trainedModel = trainModel()

#South

findWinner('Maryland', 'West Virginia', trainedModel)
findWinner('Creighton', 'NC State', trainedModel)
findWinner('Baylor', 'Santa Barbara', trainedModel)
findWinner('Missouri', 'Utah St', trainedModel)
findWinner('Arizona', 'Princeton', trainedModel)
findWinner('San Diego St', 'Col Charleston', trainedModel)
findWinner('Virginia', 'Furman', trainedModel)
findWinner('Arizona St', 'TCU', trainedModel)
#East
findWinner('Memphis', 'FL Atlantic', trainedModel)
findWinner('Duke', 'Oral Roberts', trainedModel)
findWinner('Tennessee', 'ULL', trainedModel)
findWinner('Kentucky', 'Providence', trainedModel)
findWinner('Kansas St', 'Montana St', trainedModel)
findWinner('Arizona', 'Princeton', trainedModel)
findWinner('Michigan St', 'USC', trainedModel)
findWinner('Marquette', 'Vermont', trainedModel)
findWinner('Purdue', 'Memphis', trainedModel)

#Midwest
findWinner('Houston', 'N Kentucky', trainedModel)
findWinner('Iowa', 'Auburn', trainedModel)
findWinner('Miami FL', 'Drake', trainedModel)
findWinner('Indiana', 'Kent', trainedModel)
findWinner('Iowa St', 'Pittsburgh', trainedModel)
findWinner('Mississippi St', 'Iowa St', trainedModel)
findWinner('Xavier', 'Kennesaw', trainedModel)
findWinner('Texas A&M', 'Penn St', trainedModel)
findWinner('Texas', 'Colgate', trainedModel)

#West
findWinner('Kansas', 'Howard', trainedModel)
findWinner('Arkansas', 'Illinois', trainedModel)
findWinner('VA Commonwealth', 'St Mary\'s CA', trainedModel)
findWinner('Connecticut', 'Iona', trainedModel)
findWinner('TCU', 'Arizona St', trainedModel)
findWinner('Nevada', 'TCU', trainedModel)
findWinner('Gonzaga', 'Grand Canyon', trainedModel)
findWinner('Northwestern', 'Boise St', trainedModel)
findWinner('UCLA', 'UNC Asheville', trainedModel)

print(trainedModel)

Finished run #0. Accuracy = 0.7147222222222223
Time taken: 0:03:03.382853
Finished run #1. Accuracy = 0.823125
Time taken: 0:00:00.506999
Finished run #2. Accuracy = 0.8317361111111111
Time taken: 0:00:00.522001
The average accuracy is 0.7736882716049381
Probability that Alabama wins: 0.749917599653713
Probability that West Virginia wins: 0.5089109982711089
Probability that Creighton wins: 0.8674010914462732
Probability that Santa Barbara wins: 0.5756694207676143
Probability that Missouri wins: 0.5105168255680308
Probability that Arizona wins: 0.9346843749175789
Probability that San Diego St wins: 0.7145238786930957
Probability that Virginia wins: 0.8326516860493287
Probability that Arizona St wins: 0.6332872360439272
Probability that TCU wins: 0.5872500874349055
Probability that Arizona wins: 0.5476901113599801
Probability that Alabama wins: 0.7788292379992772
Probability that FL Atlantic wins: 0.519005711014513
Probability that Duke wins: 0.7024574227554012
Probability that Tennessee

In [24]:

############################## PREDICTING THIS YEAR'S BRACKET ##############################
#round 1 v2 updated This had 27 of 32 first round games correct (2 wrong answers where major upsets, 
#3rd major upset was predicted as a close game)
def trainModel():
    model = GradientBoostingRegressor(n_estimators=350, max_depth=7, criterion='squared_error', learning_rate = 0.15,warm_start=True)
    numTrials = 3
    for i in range(numTrials):
        X_train, X_test, Y_train, Y_test = train_test_split(xTrain, yTrain, test_size= 0.25)
        startTime = datetime.now() # For some timing stuff
        results = model.fit(X_train, Y_train)
        preds = model.predict(X_test)

        preds[preds < .5] = 0
        preds[preds >= .5] = 1
        localAccuracy = np.mean(preds == Y_test)
        accuracy.append(localAccuracy)
        print ("Finished run #" + str(i) + ". Accuracy = " + str(localAccuracy))
        print ("Time taken: " + str(datetime.now() - startTime))

    if numTrials != 0:
        print ("The average accuracy is", sum(accuracy)/len(accuracy))
        
    results.fit(xTrain, yTrain)

    return model

def findWinner(team1, team2, modelUsed):
    year = [curYear]
    teamVectors = loadTeamVectors(year)[0]
    team1Vector = teamVectors[int(teams_pd[teams_pd['TeamName'] == team1].values[0][0])]
    team2Vector = teamVectors[int(teams_pd[teams_pd['TeamName'] == team2].values[0][0])]
    prediction = predictGame(team1Vector, team2Vector, 0, modelUsed)
    if (prediction < 0.5):
        print ("Probability that {0} wins: {1}".format(team2, 1 - prediction))
    else:
        print ("Probability that {0} wins: {1}".format(team1, prediction))



trainedModel = trainModel()

#South

findWinner('Maryland', 'West Virginia', trainedModel)
findWinner('Creighton', 'NC State', trainedModel)
findWinner('Baylor', 'Santa Barbara', trainedModel)
findWinner('Missouri', 'Utah St', trainedModel)
findWinner('Arizona', 'Princeton', trainedModel)
findWinner('San Diego St', 'Col Charleston', trainedModel)
findWinner('Virginia', 'Furman', trainedModel)
findWinner('Arizona St', 'TCU', trainedModel)
#East
findWinner('Memphis', 'FL Atlantic', trainedModel)
findWinner('Duke', 'Oral Roberts', trainedModel)
findWinner('Tennessee', 'ULL', trainedModel)
findWinner('Kentucky', 'Providence', trainedModel)
findWinner('Kansas St', 'Montana St', trainedModel)
findWinner('Arizona', 'Princeton', trainedModel)
findWinner('Michigan St', 'USC', trainedModel)
findWinner('Marquette', 'Vermont', trainedModel)
findWinner('Purdue', 'Memphis', trainedModel)

#Midwest
findWinner('Houston', 'N Kentucky', trainedModel)
findWinner('Iowa', 'Auburn', trainedModel)
findWinner('Miami FL', 'Drake', trainedModel)
findWinner('Indiana', 'Kent', trainedModel)
findWinner('Iowa St', 'Pittsburgh', trainedModel)
findWinner('Mississippi St', 'Iowa St', trainedModel)
findWinner('Xavier', 'Kennesaw', trainedModel)
findWinner('Texas A&M', 'Penn St', trainedModel)
findWinner('Texas', 'Colgate', trainedModel)

#West
findWinner('Kansas', 'Howard', trainedModel)
findWinner('Arkansas', 'Illinois', trainedModel)
findWinner('VA Commonwealth', 'St Mary\'s CA', trainedModel)
findWinner('Connecticut', 'Iona', trainedModel)
findWinner('TCU', 'Arizona St', trainedModel)
findWinner('Nevada', 'TCU', trainedModel)
findWinner('Gonzaga', 'Grand Canyon', trainedModel)
findWinner('Northwestern', 'Boise St', trainedModel)
findWinner('UCLA', 'UNC Asheville', trainedModel)

print(trainedModel)

Finished run #0. Accuracy = 0.7079166666666666
Time taken: 0:03:37.742974
Finished run #1. Accuracy = 0.8447222222222223
Time taken: 0:00:00.476360
Finished run #2. Accuracy = 0.8484722222222222
Time taken: 0:00:00.464996
The average accuracy is 0.7990625000000001
Probability that Maryland wins: 0.519669259821215
Probability that Creighton wins: 0.9334016907373037
Probability that Baylor wins: 0.7913323144941352
Probability that Utah St wins: 0.7443464945217086
Probability that Arizona wins: 0.837339274741882
Probability that San Diego St wins: 0.7180260122188238
Probability that Virginia wins: 0.9033514125747256
Probability that TCU wins: 0.6432903556594987
Probability that Memphis wins: 0.792114151417126
Probability that Duke wins: 0.6354451291033538
Probability that Tennessee wins: 0.6072631817582398
Probability that Kentucky wins: 0.521113887642195
Probability that Kansas St wins: 0.9001689619226573
Probability that Arizona wins: 0.837339274741882
Probability that USC wins: 0.63108

In [63]:

############################## PREDICTING THIS YEAR'S BRACKET ##############################
#Round 2 Hofstra bracket
def trainModel():
    model = GradientBoostingRegressor(n_estimators=350, max_depth=7, learning_rate = 0.1,warm_start=True)
    numTrials = 3
    for i in range(numTrials):
        X_train, X_test, Y_train, Y_test = train_test_split(xTrain, yTrain, test_size= 0.25)
        startTime = datetime.now() # For some timing stuff
        results = model.fit(X_train, Y_train)
        preds = model.predict(X_test)

        preds[preds < .5] = 0
        preds[preds >= .5] = 1
        localAccuracy = np.mean(preds == Y_test)
        accuracy.append(localAccuracy)
        print ("Finished run #" + str(i) + ". Accuracy = " + str(localAccuracy))
        print ("Time taken: " + str(datetime.now() - startTime))

    if numTrials != 0:
        print ("The average accuracy is", sum(accuracy)/len(accuracy))
        
    results.fit(xTrain, yTrain)

    return model

def findWinner(team1, team2, modelUsed):
    year = [curYear]
    teamVectors = loadTeamVectors(year)[0]
    team1Vector = teamVectors[int(teams_pd[teams_pd['TeamName'] == team1].values[0][0])]
    team2Vector = teamVectors[int(teams_pd[teams_pd['TeamName'] == team2].values[0][0])]
    prediction = predictGame(team1Vector, team2Vector, 0, modelUsed)
    if (prediction < 0.5):
        print ("Probability that {0} wins: {1}".format(team2, 1 - prediction))
    else:
        print ("Probability that {0} wins: {1}".format(team1, prediction))



trainedModel = trainModel()

#South
findWinner('Alabama', 'West Virginia', trainedModel)
findWinner('San Diego St', 'Virginia', trainedModel)
findWinner('Creighton', 'Santa Barbara', trainedModel)
findWinner('Missouri', 'Arizona', trainedModel)


#East
findWinner('Purdue', 'FL Atlantic', trainedModel)
findWinner('Duke', 'Tennessee', trainedModel)
findWinner('Kentucky', 'Kansas St', trainedModel)
findWinner('Arizona', 'Princeton', trainedModel)
findWinner('Marquette', 'USC', trainedModel)


#Midwest
findWinner('Houston', 'Iowa', trainedModel)
findWinner('Indiana', 'Drake', trainedModel)
findWinner('Texas', 'Texas A&M', trainedModel)
findWinner('Xavier', 'Iowa St', trainedModel)

#West
findWinner('Kansas', 'Arkansas', trainedModel)
findWinner('Connecticut', 'St Mary\'s CA', trainedModel)
findWinner('TCU', 'Gonzaga', trainedModel)
findWinner('UCLA', 'Boise St', trainedModel)


print(trainedModel)

Finished run #0. Accuracy = 0.7156944444444444
Time taken: 0:03:06.519591
Finished run #1. Accuracy = 0.8218055555555556
Time taken: 0:00:00.658720
Finished run #2. Accuracy = 0.82
Time taken: 0:00:00.643964
The average accuracy is 0.7767245370370369
Probability that Alabama wins: 0.7815536215830122
Probability that San Diego St wins: 0.8330123760178355
Probability that Creighton wins: 0.8254608637058897
Probability that Arizona wins: 0.8667044331523983
Probability that Purdue wins: 0.6861825668328541
Probability that Duke wins: 0.5014049390823484
Probability that Kansas St wins: 0.8230552194974288
Probability that Arizona wins: 0.906776427655781
Probability that Marquette wins: 0.8439474033026017
Probability that Houston wins: 0.7880487050082354
Probability that Indiana wins: 0.8550898290443779
Probability that Texas wins: 0.6109550324677129
Probability that Xavier wins: 0.6192514404859425
Probability that Kansas wins: 0.7897552369634523
Probability that Connecticut wins: 0.6393922654

In [25]:

############################## PREDICTING THIS YEAR'S BRACKET ##############################
#Round 3 Hofstra bracket
def trainModel():
    model = GradientBoostingRegressor(n_estimators=35, max_depth=7, learning_rate = 0.1,warm_start=True)
    numTrials = 3
    for i in range(numTrials):
        X_train, X_test, Y_train, Y_test = train_test_split(xTrain, yTrain, test_size= 0.25)
        startTime = datetime.now() # For some timing stuff
        results = model.fit(X_train, Y_train)
        preds = model.predict(X_test)

        preds[preds < .5] = 0
        preds[preds >= .5] = 1
        localAccuracy = np.mean(preds == Y_test)
        accuracy.append(localAccuracy)
        print ("Finished run #" + str(i) + ". Accuracy = " + str(localAccuracy))
        print ("Time taken: " + str(datetime.now() - startTime))

    if numTrials != 0:
        print ("The average accuracy is", sum(accuracy)/len(accuracy))
        
    results.fit(xTrain, yTrain)

    return model

def findWinner(team1, team2, modelUsed):
    year = [curYear]
    teamVectors = loadTeamVectors(year)[0]
    team1Vector = teamVectors[int(teams_pd[teams_pd['TeamName'] == team1].values[0][0])]
    team2Vector = teamVectors[int(teams_pd[teams_pd['TeamName'] == team2].values[0][0])]
    prediction = predictGame(team1Vector, team2Vector, 0, modelUsed)
    if (prediction < 0.5):
        print ("Probability that {0} wins: {1}".format(team2, 1 - prediction))
    else:
        print ("Probability that {0} wins: {1}".format(team1, prediction))



trainedModel = trainModel()

#South
findWinner('Alabama', 'San Diego St', trainedModel)
findWinner('Creighton', 'Arizona', trainedModel)



#East
findWinner('Purdue', 'Duke', trainedModel)
findWinner('Marquette', 'Kansas St', trainedModel)




#Midwest
findWinner('Houston', 'Indiana', trainedModel)
findWinner('Xavier', 'Texas', trainedModel)

#West
findWinner('Kansas', 'Connecticut', trainedModel)
findWinner('UCLA', 'Gonzaga', trainedModel)


print(trainedModel)

Finished run #0. Accuracy = 0.7339261285909713
Time taken: 0:00:12.605752
Finished run #1. Accuracy = 0.7650478796169631
Time taken: 0:00:00.056076
Finished run #2. Accuracy = 0.7633378932968536
Time taken: 0:00:00.055572
The average accuracy is 0.7554719562243503
Probability that Alabama wins: 0.6965950115926836
Probability that Arizona wins: 0.6760408819174812
Probability that Purdue wins: 0.5918531270047764
Probability that Kansas St wins: 0.9532621676819227
Probability that Houston wins: 0.853152294865706
Probability that Texas wins: 0.6011529630302059
Probability that Connecticut wins: 0.5491475512883643
Probability that UCLA wins: 0.6476673762396763
GradientBoostingRegressor(max_depth=7, n_estimators=35, warm_start=True)


In [65]:

############################## PREDICTING THIS YEAR'S BRACKET ##############################
#Round 4 Hofstra bracket
def trainModel():
    model = GradientBoostingRegressor(n_estimators=350, max_depth=7, learning_rate = 0.1,warm_start=True)
    numTrials = 3
    for i in range(numTrials):
        X_train, X_test, Y_train, Y_test = train_test_split(xTrain, yTrain, test_size= 0.25)
        startTime = datetime.now() # For some timing stuff
        results = model.fit(X_train, Y_train)
        preds = model.predict(X_test)

        preds[preds < .5] = 0
        preds[preds >= .5] = 1
        localAccuracy = np.mean(preds == Y_test)
        accuracy.append(localAccuracy)
        print ("Finished run #" + str(i) + ". Accuracy = " + str(localAccuracy))
        print ("Time taken: " + str(datetime.now() - startTime))

    if numTrials != 0:
        print ("The average accuracy is", sum(accuracy)/len(accuracy))
        
    results.fit(xTrain, yTrain)

    return model

def findWinner(team1, team2, modelUsed):
    year = [curYear]
    teamVectors = loadTeamVectors(year)[0]
    team1Vector = teamVectors[int(teams_pd[teams_pd['TeamName'] == team1].values[0][0])]
    team2Vector = teamVectors[int(teams_pd[teams_pd['TeamName'] == team2].values[0][0])]
    prediction = predictGame(team1Vector, team2Vector, 0, modelUsed)
    if (prediction < 0.5):
        print ("Probability that {0} wins: {1}".format(team2, 1 - prediction))
    else:
        print ("Probability that {0} wins: {1}".format(team1, prediction))


trainedModel = trainModel()

#South
findWinner('Alabama', 'Arizona', trainedModel)

#East Purdue doesnt move on since that game is a toss up and Duke is playing better at the moment
findWinner('Marquette', 'Duke', trainedModel)

#Midwest
findWinner('Houston', 'Texas', trainedModel)

#West # Moved Gonzaga on due to UCLA injury
findWinner('Kansas', 'Gonzaga', trainedModel)


print(trainedModel)

Finished run #0. Accuracy = 0.7136111111111111
Time taken: 0:03:44.686897
Finished run #1. Accuracy = 0.824375
Time taken: 0:00:00.622998
Finished run #2. Accuracy = 0.8264583333333333
Time taken: 0:00:00.654998
The average accuracy is 0.7805671296296297
Probability that Alabama wins: 0.7113734753366441
Probability that Marquette wins: 0.5098282664018398
Probability that Houston wins: 0.611700972999274
Probability that Gonzaga wins: 0.5255081561397896
GradientBoostingRegressor(max_depth=7, n_estimators=350, warm_start=True)


In [66]:

############################## PREDICTING THIS YEAR'S BRACKET ##############################
#Round 5/6 Hofstra bracket
def trainModel():
    model = GradientBoostingRegressor(n_estimators=350, max_depth=7, learning_rate = 0.1,warm_start=True)
    numTrials = 3
    for i in range(numTrials):
        X_train, X_test, Y_train, Y_test = train_test_split(xTrain, yTrain, test_size= 0.25)
        startTime = datetime.now() # For some timing stuff
        results = model.fit(X_train, Y_train)
        preds = model.predict(X_test)

        preds[preds < .5] = 0
        preds[preds >= .5] = 1
        localAccuracy = np.mean(preds == Y_test)
        accuracy.append(localAccuracy)
        print ("Finished run #" + str(i) + ". Accuracy = " + str(localAccuracy))
        print ("Time taken: " + str(datetime.now() - startTime))

    if numTrials != 0:
        print ("The average accuracy is", sum(accuracy)/len(accuracy))
        
    results.fit(xTrain, yTrain)

    return model

def findWinner(team1, team2, modelUsed):
    year = [curYear]
    teamVectors = loadTeamVectors(year)[0]
    team1Vector = teamVectors[int(teams_pd[teams_pd['TeamName'] == team1].values[0][0])]
    team2Vector = teamVectors[int(teams_pd[teams_pd['TeamName'] == team2].values[0][0])]
    prediction = predictGame(team1Vector, team2Vector, 0, modelUsed)
    if (prediction < 0.5):
        print ("Probability that {0} wins: {1}".format(team2, 1 - prediction))
    else:
        print ("Probability that {0} wins: {1}".format(team1, prediction))


trainedModel = trainModel()

#Final Four Prediction
findWinner('Alabama', 'Marquette', trainedModel)
findWinner('Houston', 'Gonzaga', trainedModel)

#Solving for national championship case at the same time 
findWinner('Marquette', 'Houston', trainedModel)
findWinner('Marquette', 'Gonzaga', trainedModel)


findWinner('Alabama', 'Gonzaga', trainedModel)
findWinner('Alabama', 'Houston', trainedModel)

print(trainedModel)

Finished run #0. Accuracy = 0.7147222222222223
Time taken: 0:03:04.721585
Finished run #1. Accuracy = 0.8263194444444445
Time taken: 0:00:00.514997
Finished run #2. Accuracy = 0.8245833333333333
Time taken: 0:00:00.532121
The average accuracy is 0.7817063492063493
Probability that Marquette wins: 0.638283262557428
Probability that Gonzaga wins: 0.5080667000334655
Probability that Marquette wins: 0.6012626961065163
Probability that Marquette wins: 0.7297891113963837
Probability that Gonzaga wins: 0.8475210130544192
Probability that Alabama wins: 0.7497885661245771
GradientBoostingRegressor(max_depth=7, n_estimators=350, warm_start=True)


In [61]:
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=UserWarning)

def trainModel():
    model1 = BalancedRandomForestClassifier(n_estimators=100,criterion='entropy',warm_start = True, ccp_alpha=0.015)
    model1.fit(xTrain, yTrain)
    return model1

def findWinner(team1, team2, modelUsed):
    year = [curYear]
    teamVectors = loadTeamVectors(year)[0]
    team1Vector = teamVectors[int(teams_pd[teams_pd['TeamName'] == team1].values[0][0])]
    team2Vector = teamVectors[int(teams_pd[teams_pd['TeamName'] == team2].values[0][0])]
    prediction = predictGame(team1Vector, team2Vector, 0, modelUsed)
    if (prediction < 0.5):
        print ("Probability that {0} wins: {1}".format(team2, 1 - prediction))
    else:
        print ("Probability that {0} wins: {1}".format(team1, prediction))



trainedModel = trainModel()

#South
findWinner('Alabama', 'West Virginia', trainedModel)
findWinner('Maryland', 'West Virginia', trainedModel)
findWinner('Creighton', 'NC State', trainedModel)
findWinner('Baylor', 'Santa Barbara', trainedModel)
findWinner('Missouri', 'Utah St', trainedModel)
findWinner('Arizona', 'Princeton', trainedModel)
findWinner('San Diego St', 'Col Charleston', trainedModel)
findWinner('Virginia', 'Furman', trainedModel)
findWinner('Arizona St', 'Nevada', trainedModel)
findWinner('Arizona St', 'TCU', trainedModel)
findWinner('Utah St', 'Arizona',trainedModel)
findWinner('Alabama', 'Maryland',trainedModel)
#East
findWinner('Memphis', 'FL Atlantic', trainedModel)
findWinner('Duke', 'Oral Roberts', trainedModel)
findWinner('Tennessee', 'ULL', trainedModel)
findWinner('Kentucky', 'Providence', trainedModel)
findWinner('Kansas St', 'Montana St', trainedModel)
findWinner('Arizona', 'Princeton', trainedModel)
findWinner('Michigan St', 'USC', trainedModel)
findWinner('Marquette', 'Vermont', trainedModel)
findWinner('Purdue', 'Memphis', trainedModel)

#Midwest
findWinner('Houston', 'N Kentucky', trainedModel)
findWinner('Iowa', 'Auburn', trainedModel)
findWinner('Miami FL', 'Drake', trainedModel)
findWinner('Indiana', 'Kent', trainedModel)
findWinner('Iowa St', 'Pittsburgh', trainedModel)
findWinner('Mississippi St', 'Iowa St', trainedModel)
findWinner('Xavier', 'Kennesaw', trainedModel)
findWinner('Texas A&M', 'Penn St', trainedModel)
findWinner('Texas', 'Colgate', trainedModel)

#West
findWinner('Kansas', 'Howard', trainedModel)
findWinner('Arkansas', 'Illinois', trainedModel)
findWinner('VA Commonwealth', 'St Mary\'s CA', trainedModel)
findWinner('Connecticut', 'Iona', trainedModel)
findWinner('TCU', 'Arizona St', trainedModel)
findWinner('Nevada', 'TCU', trainedModel)
findWinner('Gonzaga', 'Grand Canyon', trainedModel)
findWinner('Northwestern', 'Boise St', trainedModel)
findWinner('UCLA', 'UNC Asheville', trainedModel)

print(trainedModel)

Probability that Alabama wins: 0.7321693739875325
Probability that Maryland wins: 0.50632753389587
Probability that Creighton wins: 0.6398963433535154
Probability that Baylor wins: 0.5647703629968597
Probability that Utah St wins: 0.5824670599471551
Probability that Arizona wins: 0.7082111651331385
Probability that Col Charleston wins: 0.5058777724200392
Probability that Virginia wins: 0.5277340755948086
Probability that Arizona St wins: 0.5387190062616831
Probability that TCU wins: 0.5876605317555565
Probability that Arizona wins: 0.6419551579339292
Probability that Alabama wins: 0.7074276482587635
Probability that FL Atlantic wins: 0.5844346800190827
Probability that Oral Roberts wins: 0.5080484695122948
Probability that Tennessee wins: 0.539047561667513
Probability that Kentucky wins: 0.5547961742983827
Probability that Kansas St wins: 0.6398963433535154
Probability that Arizona wins: 0.7082111651331385
Probability that USC wins: 0.5215989551604872
Probability that Marquette wins: 0

In [37]:
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=UserWarning)
startTime = datetime.now()

def trainModel():
    model2 = BalancedRandomForestClassifier(n_estimators=100, criterion='entropy',warm_start = True, ccp_alpha=0.015)
    model2.fit(xTrain, yTrain)
    return model2

def findWinner(team1, team2, modelUsed):
    year = [curYear]
    teamVectors = loadTeamVectors(year)[0]
    team1Vector = teamVectors[int(teams_pd[teams_pd['TeamName'] == team1].values[0][0])]
    team2Vector = teamVectors[int(teams_pd[teams_pd['TeamName'] == team2].values[0][0])]
    prediction = predictGame(team1Vector, team2Vector, 0, modelUsed)
    if (prediction < 0.5):
        print ("Probability that {0} wins: {1}".format(team2, 1 - prediction))
    else:
        print ("Probability that {0} wins: {1}".format(team1, prediction))



trainedModel = trainModel()

#South
findWinner('Alabama', 'San Diego St', trainedModel)

findWinner('Creighton', 'Princeton', trainedModel)

#East
findWinner('Tennessee', 'FL Atlantic', trainedModel)
findWinner('Kansas St', 'Michigan St', trainedModel)


#Midwest
findWinner('Houston', 'Miami FL', trainedModel)
findWinner('Xavier', 'Texas', trainedModel)

#West
findWinner('Arkansas', 'Connecticut', trainedModel)
findWinner('Gonzaga', 'UCLA', trainedModel)

findWinner('Tennessee', 'Kansas St', trainedModel)
findWinner('Houston', 'Texas', trainedModel)
findWinner('Creighton', 'Alabama', trainedModel)
findWinner('Connecticut', 'UCLA', trainedModel)

findWinner('Tennessee', 'Alabama', trainedModel)
findWinner('Houston', 'UCLA', trainedModel)
findWinner('UCLA', 'Alabama', trainedModel)
findWinner('Alabama', 'Houston', trainedModel)
findWinner('Tennessee', 'UCLA', trainedModel)
print(trainedModel)

print(trainedModel)
print ("Time taken: " + str(datetime.now() - startTime))

Probability that Alabama wins: 0.679290282812651
Probability that Creighton wins: 0.6931484496458524
Probability that Tennessee wins: 0.605976060941932
Probability that Kansas St wins: 0.5935045123346455
Probability that Houston wins: 0.6913179395124147
Probability that Texas wins: 0.6478087483029713
Probability that Connecticut wins: 0.6973018613153541
Probability that UCLA wins: 0.628291913610867
Probability that Tennessee wins: 0.6465208317704882
Probability that Houston wins: 0.625793068396443
Probability that Alabama wins: 0.6870327944403241
Probability that UCLA wins: 0.6066910870231048
Probability that Alabama wins: 0.6086417281111037
Probability that Houston wins: 0.5901346887774072
Probability that Alabama wins: 0.5308092374700324
Probability that Houston wins: 0.5605028463897155
Probability that UCLA wins: 0.5569488819787474
BalancedRandomForestClassifier(ccp_alpha=0.015, criterion='entropy',
                               warm_start=True)
BalancedRandomForestClassifier(ccp_a

In [38]:
def trainModel():
    model3 = DecisionTreeClassifier (max_depth = 7)
    model3.fit(xTrain, yTrain)
    return model3

def findWinner(team1, team2, modelUsed):
    year = [curYear]
    teamVectors = loadTeamVectors(year)[0]
    team1Vector = teamVectors[int(teams_pd[teams_pd['TeamName'] == team1].values[0][0])]
    team2Vector = teamVectors[int(teams_pd[teams_pd['TeamName'] == team2].values[0][0])]
    prediction = predictGame(team1Vector, team2Vector, 0, modelUsed)
    if (prediction < 0.5):
        print ("Probability that {0} wins: {1}".format(team2, 1 - prediction))
    else:
        print ("Probability that {0} wins: {1}".format(team1, prediction))



trainedModel = trainModel()


#South
findWinner('Alabama', 'West Virginia', trainedModel)
findWinner('Maryland', 'West Virginia', trainedModel)
findWinner('Creighton', 'NC State', trainedModel)
findWinner('Baylor', 'Santa Barbara', trainedModel)
findWinner('Missouri', 'Utah St', trainedModel)
findWinner('Arizona', 'Princeton', trainedModel)
findWinner('San Diego St', 'Col Charleston', trainedModel)
findWinner('Virginia', 'Furman', trainedModel)
findWinner('Arizona St', 'Nevada', trainedModel)
findWinner('Arizona St', 'TCU', trainedModel)
findWinner('Utah St', 'Arizona',trainedModel)
findWinner('Alabama', 'Maryland',trainedModel)
#East
findWinner('Memphis', 'FL Atlantic', trainedModel)
findWinner('Duke', 'Oral Roberts', trainedModel)
findWinner('Tennessee', 'ULL', trainedModel)
findWinner('Kentucky', 'Providence', trainedModel)
findWinner('Kansas St', 'Montana St', trainedModel)
findWinner('Arizona', 'Princeton', trainedModel)
findWinner('Michigan St', 'USC', trainedModel)
findWinner('Marquette', 'Vermont', trainedModel)
findWinner('Purdue', 'Memphis', trainedModel)

#Midwest
findWinner('Houston', 'N Kentucky', trainedModel)
findWinner('Iowa', 'Auburn', trainedModel)
findWinner('Miami FL', 'Drake', trainedModel)
findWinner('Indiana', 'Kent', trainedModel)
findWinner('Iowa St', 'Pittsburgh', trainedModel)
findWinner('Mississippi St', 'Iowa St', trainedModel)
findWinner('Xavier', 'Kennesaw', trainedModel)
findWinner('Texas A&M', 'Penn St', trainedModel)
findWinner('Texas', 'Colgate', trainedModel)

#West
findWinner('Kansas', 'Howard', trainedModel)
findWinner('Arkansas', 'Illinois', trainedModel)
findWinner('VA Commonwealth', 'St Mary\'s CA', trainedModel)
findWinner('Connecticut', 'Iona', trainedModel)
findWinner('TCU', 'Arizona St', trainedModel)
findWinner('Nevada', 'TCU', trainedModel)
findWinner('Gonzaga', 'Grand Canyon', trainedModel)
findWinner('Northwestern', 'Boise St', trainedModel)
findWinner('UCLA', 'UNC Asheville', trainedModel)

print(trainedModel)
print ("Time taken: " + str(datetime.now() - startTime))

Probability that Alabama wins: 0.8386660657954034
Probability that Maryland wins: 0.6811955168119551
Probability that Creighton wins: 0.9127906976744186
Probability that Baylor wins: 0.8336653386454184
Probability that Utah St wins: 0.5695322376738305
Probability that Arizona wins: 0.8336653386454184
Probability that San Diego St wins: 0.7378516624040921
Probability that Virginia wins: 0.7378516624040921
Probability that Arizona St wins: 0.628158844765343
Probability that TCU wins: 0.5984807379272925
Probability that Arizona wins: 0.5984807379272925
Probability that Alabama wins: 0.8386660657954034
Probability that FL Atlantic wins: 0.5134189031505251
Probability that Duke wins: 0.7378516624040921
Probability that Tennessee wins: 0.9127906976744186
Probability that Kentucky wins: 0.628158844765343
Probability that Kansas St wins: 0.9127906976744186
Probability that Arizona wins: 0.8336653386454184
Probability that USC wins: 0.5134189031505251
Probability that Vermont wins: 0.8900747065

In [58]:
def trainModel():
    model4 = GradientBoostingRegressor(n_estimators=100, max_depth=7, learning_rate = 0.1,warm_start=True)
    model4.fit(xTrain, yTrain)
    return model4

def findWinner(team1, team2, modelUsed):
    year = [curYear]
    teamVectors = loadTeamVectors(year)[0]
    team1Vector = teamVectors[int(teams_pd[teams_pd['TeamName'] == team1].values[0][0])]
    team2Vector = teamVectors[int(teams_pd[teams_pd['TeamName'] == team2].values[0][0])]
    prediction = predictGame(team1Vector, team2Vector, 0, modelUsed)
    if (prediction < 0.5):
        print ("Probability that {0} wins: {1}".format(team2, 1 - prediction))
    else:
        print ("Probability that {0} wins: {1}".format(team1, prediction))



trainedModel = trainModel()

#South
findWinner('Alabama', 'West Virginia', trainedModel)
findWinner('Maryland', 'West Virginia', trainedModel)
findWinner('Creighton', 'NC State', trainedModel)
findWinner('Baylor', 'Santa Barbara', trainedModel)
findWinner('Missouri', 'Utah St', trainedModel)
findWinner('Arizona', 'Princeton', trainedModel)
findWinner('San Diego St', 'Col Charleston', trainedModel)
findWinner('Virginia', 'Furman', trainedModel)
findWinner('Arizona St', 'Nevada', trainedModel)
findWinner('Arizona St', 'TCU', trainedModel)
findWinner('Utah St', 'Arizona',trainedModel)
findWinner('Alabama', 'Maryland',trainedModel)
#East
findWinner('Memphis', 'FL Atlantic', trainedModel)
findWinner('Duke', 'Oral Roberts', trainedModel)
findWinner('Tennessee', 'ULL', trainedModel)
findWinner('Kentucky', 'Providence', trainedModel)
findWinner('Kansas St', 'Montana St', trainedModel)
findWinner('Arizona', 'Princeton', trainedModel)
findWinner('Michigan St', 'USC', trainedModel)
findWinner('Marquette', 'Vermont', trainedModel)
findWinner('Purdue', 'Memphis', trainedModel)

#Midwest
findWinner('Houston', 'N Kentucky', trainedModel)
findWinner('Iowa', 'Auburn', trainedModel)
findWinner('Miami FL', 'Drake', trainedModel)
findWinner('Indiana', 'Kent', trainedModel)
findWinner('Iowa St', 'Pittsburgh', trainedModel)
findWinner('Mississippi St', 'Iowa St', trainedModel)
findWinner('Xavier', 'Kennesaw', trainedModel)
findWinner('Texas A&M', 'Penn St', trainedModel)
findWinner('Texas', 'Colgate', trainedModel)

#West
findWinner('Kansas', 'Howard', trainedModel)
findWinner('Arkansas', 'Illinois', trainedModel)
findWinner('VA Commonwealth', 'St Mary\'s CA', trainedModel)
findWinner('Connecticut', 'Iona', trainedModel)
findWinner('TCU', 'Arizona St', trainedModel)
findWinner('Nevada', 'TCU', trainedModel)
findWinner('Gonzaga', 'Grand Canyon', trainedModel)
findWinner('Northwestern', 'Boise St', trainedModel)
findWinner('UCLA', 'UNC Asheville', trainedModel)


Probability that Alabama wins: 0.7805087005879715
Probability that West Virginia wins: 0.5369707696970941
Probability that Creighton wins: 0.9653582378201253
Probability that Baylor wins: 0.8491078529010659
Probability that Utah St wins: 0.5292998270450044
Probability that Arizona wins: 0.9222952936808021
Probability that San Diego St wins: 0.7810604820757862
Probability that Virginia wins: 0.9344288814857353
Probability that Nevada wins: 0.5148464343380099
Probability that TCU wins: 0.6406067689278474
Probability that Arizona wins: 0.7177962927201764
Probability that Alabama wins: 0.8133686083576948
Probability that FL Atlantic wins: 0.6110219484262627
Probability that Duke wins: 0.7148692771524122
Probability that Tennessee wins: 0.8867672071413268
Probability that Kentucky wins: 0.5751819460095409
Probability that Kansas St wins: 0.9732568484266229
Probability that Arizona wins: 0.9222952936808021
Probability that USC wins: 0.5983821029056275
Probability that Marquette wins: 0.95358

In [17]:
############################## PREDICTING THIS YEAR'S BRACKET NIT ##############################
#round 1 hoftstra bracket
def trainModel():
    model = GradientBoostingRegressor(n_estimators=100, max_depth=5, learning_rate = 0.1,warm_start=True)
    numTrials = 3
    for i in range(numTrials):
        X_train, X_test, Y_train, Y_test = train_test_split(xTrain, yTrain, test_size= 0.25)
        startTime = datetime.now() # For some timing stuff
        results = model.fit(X_train, Y_train)
        preds = model.predict(X_test)

        preds[preds < .5] = 0
        preds[preds >= .5] = 1
        localAccuracy = np.mean(preds == Y_test)
        accuracy.append(localAccuracy)
        print ("Finished run #" + str(i) + ". Accuracy = " + str(localAccuracy))
        print ("Time taken: " + str(datetime.now() - startTime))

    if numTrials != 0:
        print ("The average accuracy is", sum(accuracy)/len(accuracy))
        
    results.fit(xTrain, yTrain)

    return model

def findWinner(team1, team2, modelUsed):
    year = [curYear]
    teamVectors = loadTeamVectors(year)[0]
    team1Vector = teamVectors[int(teams_pd[teams_pd['TeamName'] == team1].values[0][0])]
    team2Vector = teamVectors[int(teams_pd[teams_pd['TeamName'] == team2].values[0][0])]
    prediction = predictGame(team1Vector, team2Vector, 0, modelUsed)
    if (prediction < 0.5):
        print ("Probability that {0} wins: {1}".format(team2, 1 - prediction))
    else:
        print ("Probability that {0} wins: {1}".format(team1, prediction))



trainedModel = trainModel()

#South
findWinner('Hofstra', 'Rutgers', trainedModel)
findWinner('Cincinnati', 'Virginia Tech', trainedModel)
findWinner('New Mexico', 'Utah Valley', trainedModel)
findWinner('Colorado', 'Seton Hall', trainedModel)
findWinner('Clemson', 'Morehead St', trainedModel)
findWinner('UAB', 'Southern Miss', trainedModel)
findWinner('Vanderbilt', 'Yale', trainedModel)
findWinner('Michigan', 'Toledo', trainedModel)
findWinner('Oklahoma St', 'Youngstown St', trainedModel)
findWinner('Washington St', 'E Washington', trainedModel)
findWinner('North Texas', 'Alcorn St',trainedModel)
findWinner('Sam Houston St', 'Santa Clara',trainedModel)
#East
findWinner('Oregon', 'UC Irvine', trainedModel)
findWinner('Florida', 'UCF', trainedModel)
findWinner('Wisconsin', 'Bradley', trainedModel)
findWinner('Liberty', 'Villanova', trainedModel)

print(trainedModel)

Finished run #0. Accuracy = 0.7304861111111111
Time taken: 0:00:38.153336
Finished run #1. Accuracy = 0.7520138888888889
Time taken: 0:00:00.123000
Finished run #2. Accuracy = 0.7495833333333334
Time taken: 0:00:00.117999
The average accuracy is 0.7570949074074075
Probability that Rutgers wins: 0.6666499160558038
Probability that Cincinnati wins: 0.5445159179520229
Probability that New Mexico wins: 0.6600602759094669
Probability that Colorado wins: 0.5126225904663186
Probability that Clemson wins: 0.9105920966031816
Probability that UAB wins: 0.6048359868969186
Probability that Vanderbilt wins: 0.5973419556283218
Probability that Michigan wins: 0.7368534886794725
Probability that Oklahoma St wins: 0.7732042513620405
Probability that Washington St wins: 0.6540562710287351
Probability that North Texas wins: 0.9029530839834512
Probability that Santa Clara wins: 0.600011394939595
Probability that Oregon wins: 0.6824604776836871
Probability that Florida wins: 0.525889066741352
Probability t

In [ ]:
############################## PREDICTING THIS YEAR'S BRACKET NIT ##############################
#round 1 hoftstra bracket
def trainModel():
    model = GradientBoostingRegressor(n_estimators=300, max_depth=7, learning_rate = 0.15, warm_start=True)
    numTrials = 3
    for i in range(numTrials):
        X_train, X_test, Y_train, Y_test = train_test_split(xTrain, yTrain, test_size= 0.25)
        startTime = datetime.now() # For some timing stuff
        results = model.fit(X_train, Y_train)
        preds = model.predict(X_test)

        preds[preds < .5] = 0
        preds[preds >= .5] = 1
        localAccuracy = np.mean(preds == Y_test)
        accuracy.append(localAccuracy)
        print ("Finished run #" + str(i) + ". Accuracy = " + str(localAccuracy))
        print ("Time taken: " + str(datetime.now() - startTime))

    if numTrials != 0:
        print ("The average accuracy is", sum(accuracy)/len(accuracy))
        
    results.fit(xTrain, yTrain)

    return model

def findWinner(team1, team2, modelUsed):
    year = [curYear]
    teamVectors = loadTeamVectors(year)[0]
    team1Vector = teamVectors[int(teams_pd[teams_pd['TeamName'] == team1].values[0][0])]
    team2Vector = teamVectors[int(teams_pd[teams_pd['TeamName'] == team2].values[0][0])]
    prediction = predictGame(team1Vector, team2Vector, 0, modelUsed)
    if (prediction < 0.5):
        print ("Probability that {0} wins: {1}".format(team2, 1 - prediction))
    else:
        print ("Probability that {0} wins: {1}".format(team1, prediction))



trainedModel = trainModel()

#South
findWinner('Hofstra', 'Virginia Tech', trainedModel)
findWinner('Cincinnati', 'Hofstra', trainedModel)


findWinner('Vanderbilt', 'Michigan', trainedModel)



findWinner('Liberty', 'Wisconsin', trainedModel)

print(trainedModel)

In [26]:
############################## PREDICTING THIS YEAR'S BRACKET NIT ##############################
#second chance bracket with updated stats
def trainModel():
    model = GradientBoostingRegressor(n_estimators=350, max_depth=7, criterion='squared_error', learning_rate = 0.15,warm_start=True)
    numTrials = 3
    for i in range(numTrials):
        X_train, X_test, Y_train, Y_test = train_test_split(xTrain, yTrain, test_size= 0.25)
        startTime = datetime.now() # For some timing stuff
        results = model.fit(X_train, Y_train)
        preds = model.predict(X_test)

        preds[preds < .5] = 0
        preds[preds >= .5] = 1
        localAccuracy = np.mean(preds == Y_test)
        accuracy.append(localAccuracy)
        print ("Finished run #" + str(i) + ". Accuracy = " + str(localAccuracy))
        print ("Time taken: " + str(datetime.now() - startTime))

    if numTrials != 0:
        print ("The average accuracy is", sum(accuracy)/len(accuracy))
        
    results.fit(xTrain, yTrain)

    return model

def findWinner(team1, team2, modelUsed):
    year = [curYear]
    teamVectors = loadTeamVectors(year)[0]
    team1Vector = teamVectors[int(teams_pd[teams_pd['TeamName'] == team1].values[0][0])]
    team2Vector = teamVectors[int(teams_pd[teams_pd['TeamName'] == team2].values[0][0])]
    prediction = predictGame(team1Vector, team2Vector, 0, modelUsed)
    if (prediction < 0.5):
        print ("Probability that {0} wins: {1}".format(team2, 1 - prediction))
    else:
        print ("Probability that {0} wins: {1}".format(team1, prediction))



trainedModel = trainModel()

#South
findWinner('Alabama', 'San Diego St', trainedModel)

findWinner('Creighton', 'Princeton', trainedModel)

#East
findWinner('Tennessee', 'FL Atlantic', trainedModel)
findWinner('Kansas St', 'Michigan St', trainedModel)


#Midwest
findWinner('Houston', 'Miami FL', trainedModel)
findWinner('Xavier', 'Texas', trainedModel)

#West
findWinner('Arkansas', 'Connecticut', trainedModel)
findWinner('Gonzaga', 'UCLA', trainedModel)

print(trainedModel)

Finished run #0. Accuracy = 0.6987004103967168
Time taken: 0:02:12.145696
Finished run #1. Accuracy = 0.8528556771545828
Time taken: 0:00:00.298995
Finished run #2. Accuracy = 0.8446477428180574
Time taken: 0:00:00.313994
The average accuracy is 0.7698928408572733
Probability that Alabama wins: 0.5992119334609178
Probability that Creighton wins: 0.858333848906545
Probability that Tennessee wins: 0.6068794203944805
Probability that Kansas St wins: 0.802643786119209
Probability that Houston wins: 0.653393209619825
Probability that Texas wins: 0.6484340630485782
Probability that Connecticut wins: 0.6775122803570313
Probability that UCLA wins: 0.7463461508255687
GradientBoostingRegressor(criterion='squared_error', learning_rate=0.15,
                          max_depth=7, n_estimators=350, warm_start=True)


In [27]:
############################## PREDICTING THIS YEAR'S BRACKET NIT ##############################
#round 1 hoftstra bracket
def trainModel():
    model = GradientBoostingRegressor(n_estimators=350, max_depth=7, criterion='squared_error', learning_rate = 0.15,warm_start=True)
    numTrials = 3
    for i in range(numTrials):
        X_train, X_test, Y_train, Y_test = train_test_split(xTrain, yTrain, test_size= 0.25)
        startTime = datetime.now() # For some timing stuff
        results = model.fit(X_train, Y_train)
        preds = model.predict(X_test)

        preds[preds < .5] = 0
        preds[preds >= .5] = 1
        localAccuracy = np.mean(preds == Y_test)
        accuracy.append(localAccuracy)
        print ("Finished run #" + str(i) + ". Accuracy = " + str(localAccuracy))
        print ("Time taken: " + str(datetime.now() - startTime))

    if numTrials != 0:
        print ("The average accuracy is", sum(accuracy)/len(accuracy))
        
    results.fit(xTrain, yTrain)

    return model

def findWinner(team1, team2, modelUsed):
    year = [curYear]
    teamVectors = loadTeamVectors(year)[0]
    team1Vector = teamVectors[int(teams_pd[teams_pd['TeamName'] == team1].values[0][0])]
    team2Vector = teamVectors[int(teams_pd[teams_pd['TeamName'] == team2].values[0][0])]
    prediction = predictGame(team1Vector, team2Vector, 0, modelUsed)
    if (prediction < 0.5):
        print ("Probability that {0} wins: {1}".format(team2, 1 - prediction))
    else:
        print ("Probability that {0} wins: {1}".format(team1, prediction))



trainedModel = trainModel()

#South
findWinner('Alabama', 'Creighton', trainedModel)

#East
findWinner('Tennessee', 'Kansas St', trainedModel)


#Midwest
findWinner('Houston', 'Texas', trainedModel)
#West
findWinner('UCLA', 'Connecticut', trainedModel)


print(trainedModel)

Finished run #0. Accuracy = 0.7069083447332422
Time taken: 0:02:21.399766
Finished run #1. Accuracy = 0.8544801641586868
Time taken: 0:00:00.317824
Finished run #2. Accuracy = 0.8535396716826266
Time taken: 0:00:00.344967
The average accuracy is 0.7786636456908346
Probability that Alabama wins: 0.7033037381122583
Probability that Tennessee wins: 0.5427341304838418
Probability that Texas wins: 0.5052168205975943
Probability that UCLA wins: 0.7493563572017574
GradientBoostingRegressor(criterion='squared_error', learning_rate=0.15,
                          max_depth=7, n_estimators=350, warm_start=True)


In [28]:
############################## PREDICTING THIS YEAR'S BRACKET NIT ##############################
#round 1 hoftstra bracket
def trainModel():
    model = GradientBoostingRegressor(n_estimators=350, max_depth=7, criterion='squared_error', learning_rate = 0.15,warm_start=True)
    numTrials = 3
    for i in range(numTrials):
        X_train, X_test, Y_train, Y_test = train_test_split(xTrain, yTrain, test_size= 0.25)
        startTime = datetime.now() # For some timing stuff
        results = model.fit(X_train, Y_train)
        preds = model.predict(X_test)

        preds[preds < .5] = 0
        preds[preds >= .5] = 1
        localAccuracy = np.mean(preds == Y_test)
        accuracy.append(localAccuracy)
        print ("Finished run #" + str(i) + ". Accuracy = " + str(localAccuracy))
        print ("Time taken: " + str(datetime.now() - startTime))

    if numTrials != 0:
        print ("The average accuracy is", sum(accuracy)/len(accuracy))
        
    results.fit(xTrain, yTrain)

    return model

def findWinner(team1, team2, modelUsed):
    year = [curYear]
    teamVectors = loadTeamVectors(year)[0]
    team1Vector = teamVectors[int(teams_pd[teams_pd['TeamName'] == team1].values[0][0])]
    team2Vector = teamVectors[int(teams_pd[teams_pd['TeamName'] == team2].values[0][0])]
    prediction = predictGame(team1Vector, team2Vector, 0, modelUsed)
    if (prediction < 0.5):
        print ("Probability that {0} wins: {1}".format(team2, 1 - prediction))
    else:
        print ("Probability that {0} wins: {1}".format(team1, prediction))



trainedModel = trainModel()

#South
findWinner('Alabama', 'Tennessee', trainedModel)


#Midwest
findWinner('UCLA', 'Texas', trainedModel)
#West
findWinner('UCLA', 'Tennessee', trainedModel)
findWinner('UCLA', 'Alabama', trainedModel)
findWinner('Texas', 'Tennessee', trainedModel)
findWinner('Texas', 'Alabama', trainedModel)

print(trainedModel)

Finished run #0. Accuracy = 0.7109268125854993
Time taken: 0:02:29.942533
Finished run #1. Accuracy = 0.853967168262654
Time taken: 0:00:00.402022
Finished run #2. Accuracy = 0.8527701778385773
Time taken: 0:00:00.430991
The average accuracy is 0.7841085271317831
Probability that Tennessee wins: 0.5525388736155286
Probability that UCLA wins: 0.5130036430566518
Probability that UCLA wins: 0.7656507563242589
Probability that UCLA wins: 0.6209908602679503
Probability that Texas wins: 0.7244414142722702
Probability that Texas wins: 0.7008445567776312
GradientBoostingRegressor(criterion='squared_error', learning_rate=0.15,
                          max_depth=7, n_estimators=350, warm_start=True)


In [18]:
############################## PREDICTING THIS YEAR'S BRACKET NIT ##############################
#round 1 hoftstra bracket
def trainModel():
    model = GradientBoostingRegressor(n_estimators=100,warm_start=True)
    numTrials = 3
    for i in range(numTrials):
        X_train, X_test, Y_train, Y_test = train_test_split(xTrain, yTrain, test_size= 0.33)
        startTime = datetime.now() # For some timing stuff
        results = model.fit(X_train, Y_train)
        preds = model.predict(X_test)

        preds[preds < .5] = 0
        preds[preds >= .5] = 1
        localAccuracy = np.mean(preds == Y_test)
        accuracy.append(localAccuracy)
        print ("Finished run #" + str(i) + ". Accuracy = " + str(localAccuracy))
        print ("Time taken: " + str(datetime.now() - startTime))

    if numTrials != 0:
        print ("The average accuracy is", sum(accuracy)/len(accuracy))
        
    results.fit(xTrain, yTrain)

    return model

def findWinner(team1, team2, modelUsed):
    year = [curYear]
    teamVectors = loadTeamVectors(year)[0]
    team1Vector = teamVectors[int(teams_pd[teams_pd['TeamName'] == team1].values[0][0])]
    team2Vector = teamVectors[int(teams_pd[teams_pd['TeamName'] == team2].values[0][0])]
    prediction = predictGame(team1Vector, team2Vector, 0, modelUsed)
    if (prediction < 0.5):
        print ("Probability that {0} wins: {1}".format(team2, 1 - prediction))
    else:
        print ("Probability that {0} wins: {1}".format(team1, prediction))



trainedModel = trainModel()

#South
findWinner('Hofstra', 'Rutgers', trainedModel)
findWinner('Cincinnati', 'Virginia Tech', trainedModel)
findWinner('New Mexico', 'Utah Valley', trainedModel)
findWinner('Colorado', 'Seton Hall', trainedModel)
findWinner('Clemson', 'Morehead St', trainedModel)
findWinner('UAB', 'Southern Miss', trainedModel)
findWinner('Vanderbilt', 'Yale', trainedModel)
findWinner('Michigan', 'Toledo', trainedModel)
findWinner('Oklahoma St', 'Youngstown St', trainedModel)
findWinner('Washington St', 'E Washington', trainedModel)
findWinner('North Texas', 'Alcorn St',trainedModel)
findWinner('Sam Houston St', 'Santa Clara',trainedModel)
#East
findWinner('Oregon', 'UC Irvine', trainedModel)
findWinner('Florida', 'UCF', trainedModel)
findWinner('Wisconsin', 'Bradley', trainedModel)
findWinner('Liberty', 'Villanova', trainedModel)

print(trainedModel)

Finished run #0. Accuracy = 0.7333754208754208
Time taken: 0:00:15.324004
Finished run #1. Accuracy = 0.7440551346801347
Time taken: 0:00:00.090000
Finished run #2. Accuracy = 0.7399515993265994
Time taken: 0:00:00.090438
The average accuracy is 0.754330808080808
Probability that Rutgers wins: 0.7252865690964534
Probability that Cincinnati wins: 0.5382500006890597
Probability that New Mexico wins: 0.6124527687021065
Probability that Colorado wins: 0.50578706093555
Probability that Clemson wins: 0.8592858607431355
Probability that UAB wins: 0.5856774823319273
Probability that Vanderbilt wins: 0.5627871905514236
Probability that Michigan wins: 0.6117672753203892
Probability that Oklahoma St wins: 0.8111753280830042
Probability that Washington St wins: 0.7754578448978251
Probability that North Texas wins: 0.9031573264683423
Probability that Santa Clara wins: 0.5472648422557995
Probability that Oregon wins: 0.7440398828590052
Probability that Florida wins: 0.5459062555527877
Probability th

In [ ]:
Finished run #0. Accuracy = 0.6704166666666667
Time taken: 0:02:47.408250
Finished run #1. Accuracy = 0.8723611111111111
Time taken: 0:00:00.375423
Finished run #2. Accuracy = 0.8670833333333333
Time taken: 0:00:00.375972
The average accuracy is 0.7626639660493827
Probability that Rutgers wins: 0.8755464323567066
Probability that Cincinnati wins: 0.7988674343570674
Probability that New Mexico wins: 0.9938119552029164
Probability that Colorado wins: 0.5511015749563652
Probability that Clemson wins: 0.9992730244567153
Probability that UAB wins: 0.5332057843757134
Probability that Vanderbilt wins: 0.9021854830335755
Probability that Michigan wins: 1.4784323885694186
Probability that Oklahoma St wins: 1.0789395693349941
Probability that Washington St wins: 1.0186825692981716
Probability that North Texas wins: 0.920277408675058
Probability that Santa Clara wins: 0.8376314588271232
Probability that Oregon wins: 0.8652076433125271
Probability that Florida wins: 0.7117628201779037
Probability that Wisconsin wins: 1.0247198443459264
Probability that Villanova wins: 0.6390132532168182
GradientBoostingRegressor(learning_rate=0.5, max_depth=7, n_estimators=300,
                          warm_start=True)